# Building a csv of all your PBs
-- a short story by baldnate

# Get user id

In [1]:
import json
import pandas as pd
import requests
import math

users = {}
def getUserId(username):
  if username not in users:
    url = "https://www.speedrun.com/api/v1/users?name=" + username
    data = requests.get(url).json()['data']
    if len(data) == 1:
      users[username] = data[0]['id']
    else:
      raise Exception('Searched for ' + username + ', got back ' + str(len(data)) + ' entries (expected 1)') 
  return users[username]

userid = getUserId('baldnate')

# Get PBs

In [2]:
def getPBs(userid, all = False):
  url = "https://www.speedrun.com/api/v1/users/" + userid + "/personal-bests?embed=game,category,region,platform,players"
  data = requests.get(url).json()['data']
  pbdf = pd.DataFrame(data)
  pbdf = pbdf.join(pbdf['run'].apply(pd.Series), rsuffix='run')
  pbdf.drop(axis=1, columns=['run'], inplace=True)
  if all:
    allurl = "https://www.speedrun.com/api/v1/runs?user=" + userid + "&embed=game,category,region,platform,players"
    data = requests.get(allurl).json()['data']
    alldf = pd.DataFrame(data)
    alldf['place'] = math.nan
    pbdf = pbdf.append(alldf)
    return pbdf
  return pbdf

rawdf = getPBs(userid, all=True)
runsdf = pd.DataFrame()

In [3]:
rawdf.tail()

,place,game,category,players,region,platform,id,weblink,gamerun,level,...,comment,status,playersrun,date,submitted,times,system,splits,values,links
15,NaN,"{'data': {'id': '4d79jz17', 'names': {'interna...","{'data': {'id': 'wdmw8x2q', 'name': 'Any%', 'w...","{'data': [{'rel': 'user', 'id': 'kj9yg2wx', 'n...","{'data': {'id': 'pr184lqn', 'name': 'USA / NTS...","{'data': {'id': 'jm95z9ol', 'name': 'Nintendo ...",zn824dvz,https://www.speedrun.com/stridernes/run/zn824dvz,NaN,None,...,"No air zip, no yggdrasil quick kill... just re...","{'status': 'verified', 'examiner': 'qjorq7j6',...",NaN,2018-01-12,2018-01-13T07:01:07Z,"{'primary': 'PT4M0.420S', 'primary_t': 240.42,...","{'platform': 'jm95z9ol', 'emulated': True, 're...",None,{},"[{'rel': 'self', 'uri': 'https://www.speedrun...."
16,NaN,"{'data': {'id': '4d79jz17', 'names': {'interna...","{'data': {'id': 'wk6po6xd', 'name': 'Zipless',...","{'data': [{'rel': 'user', 'id': 'kj9yg2wx', 'n...","{'data': {'id': 'pr184lqn', 'name': 'USA / NTS...","{'data': {'id': 'jm95z9ol', 'name': 'Nintendo ...",z510n5jm,https://www.speedrun.com/stridernes/run/z510n5jm,NaN,None,...,Played on BizHawk 2.3.0,"{'status': 'verified', 'examiner': 'qjorq7j6',...",NaN,2018-08-31,2018-08-31T20:14:04Z,"{'primary': 'PT29M0.770S', 'primary_t': 1740.7...","{'platform': 'jm95z9ol', 'emulated': True, 're...","{'rel': 'splits.io', 'uri': 'https://splits.io...",{},"[{'rel': 'self', 'uri': 'https://www.speedrun...."
17,NaN,"{'data': {'id': '4d79jz17', 'names': {'interna...","{'data': {'id': 'wdmw8x2q', 'name': 'Any%', 'w...","{'data': [{'rel': 'user', 'id': 'kj9yg2wx', 'n...","{'data': {'id': 'pr184lqn', 'name': 'USA / NTS...","{'data': {'id': 'jm95z9ol', 'name': 'Nintendo ...",mekjj90m,https://www.speedrun.com/stridernes/run/mekjj90m,NaN,None,...,None,"{'status': 'verified', 'examiner': '18q3v7jn',...",NaN,2017-10-01,2017-10-01T17:32:25Z,"{'primary': 'PT4M44.840S', 'primary_t': 284.84...","{'platform': 'jm95z9ol', 'emulated': True, 're...",None,{},"[{'rel': 'self', 'uri': 'https://www.speedrun...."
18,NaN,"{'data': {'id': '4d79jz17', 'names': {'interna...","{'data': {'id': 'wdmw8x2q', 'name': 'Any%', 'w...","{'data': [{'rel': 'user', 'id': 'kj9yg2wx', 'n...","{'data': {'id': 'pr184lqn', 'name': 'USA / NTS...","{'data': {'id': 'jm95z9ol', 'name': 'Nintendo ...",y672xx0m,https://www.speedrun.com/stridernes/run/y672xx0m,NaN,None,...,First PB after switching to fast Red Dragon zi...,"{'status': 'verified', 'examiner': '18q3v7jn',...",NaN,2017-12-18,2017-12-18T19:03:21Z,"{'primary': 'PT4M25.390S', 'primary_t': 265.39...","{'platform': 'jm95z9ol', 'emulated': True, 're...",None,{},"[{'rel': 'self', 'uri': 'https://www.speedrun...."
19,NaN,"{'data': {'id': '4d79jz17', 'names': {'interna...","{'data': {'id': 'wdmw8x2q', 'name': 'Any%', 'w...","{'data': [{'rel': 'user', 'id': 'kj9yg2wx', 'n...","{'data': {'id': 'pr184lqn', 'name': 'USA / NTS...","{'data': {'id': 'jm95z9ol', 'name': 'Nintendo ...",y8qwep5y,https://www.speedrun.com/stridernes/run/y8qwep5y,NaN,None,...,"got the air zip. no screen wrap, no yggy quick...","{'status': 'verified', 'examiner': '18q3v7jn',...",NaN,2018-01-15,2018-01-16T04:56:50Z,"{'primary': 'PT3M53.990S', 'primary_t': 233.99...","{'platform': 'jm95z9ol', 'emulated': True, 're...",None,{},"[{'rel': 'self', 'uri': 'https://www.speedrun...."


# Co-Op - aka: write player(s) to a column

# "Simple" Columns

In [4]:
runsdf['place'] = rawdf['place']
runsdf['gamename'] = rawdf.apply(lambda x: x.game['data']['names']['international'], axis=1)
runsdf['categoryname'] = rawdf.apply(lambda x: x.category['data']['name'], axis=1)
runsdf['time'] = rawdf.apply(lambda x: x.times['primary_t'], axis=1)
runsdf['date'] = rawdf.apply(lambda x: x.date, axis=1)
runsdf['video'] = rawdf.apply(lambda x: x.videos['links'][0]['uri'], axis=1)
runsdf['comment'] = rawdf.apply(lambda x: str(x.comment).replace('\n', ' ').replace('\r', ' ') , axis=1)

# Columns that need optional handling

In [5]:
def getRegion(x):
  if len(x.region['data']) == 0:
    return "None"
  else:
    return x.region['data']['name']

runsdf['regionname'] = rawdf.apply(lambda x: getRegion(x), axis=1)


In [6]:
def getPlatform(x):
  if len(x.platform['data']) == 0:
    return "None"
  else:
    return x.platform['data']['name']

runsdf['platformname'] = rawdf.apply(lambda x: getPlatform(x), axis=1)

# Sub-Categories

Memoized for speed and kindness.

In [7]:
varMemo = {}

In [8]:
def getVariable(variableid):
  if variableid not in varMemo:
    url = "https://www.speedrun.com/api/v1/variables/" + variableid
    response = requests.get(url)
    varMemo[variableid] = response.json()['data']
  return varMemo[variableid]

def getValue(variableid, valueid):
  var = getVariable(variableid)
  return var['values']['values'][valueid]['label']

def getSubCategories(x):
  if x['values'] == {}:
    return "None"
  else:
    vals = []
    for varid, valid in x['values'].items():
      if getVariable(varid)['is-subcategory']:
        vals.append(getValue(varid, valid))  
    return " -- ".join(vals)

runsdf['subcategories'] = rawdf.apply(lambda x: getSubCategories(x), axis=1)

# Dump to a csv

In [9]:
def getPlayers(x):
  players = []
  for p in x.players['data']:
    players.append(p['names']['international']) 
  return ", ".join(players)

runsdf['players'] = rawdf.apply(lambda x: getPlayers(x), axis=1)

In [10]:
import csv
runsdf.fillna(value="").to_csv('runs.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
print('csv exported')

csv exported


In [11]:
csvdf = pd.read_csv('runs.csv')
csvdf.tail()

,place,gamename,categoryname,time,date,video,comment,regionname,platformname,subcategories,players
452,NaN,Strider (NES),Any%,240.42,2018-01-12,https://www.twitch.tv/videos/218086595,"No air zip, no yggdrasil quick kill... just re...",USA / NTSC,Nintendo Entertainment System,None,baldnate
453,NaN,Strider (NES),Zipless,1740.77,2018-08-31,https://www.youtube.com/watch?v=xAW9a_Lp3Sk,Played on BizHawk 2.3.0,USA / NTSC,Nintendo Entertainment System,None,baldnate
454,NaN,Strider (NES),Any%,284.84,2017-10-01,https://www.twitch.tv/videos/178950739,None,USA / NTSC,Nintendo Entertainment System,None,baldnate
455,NaN,Strider (NES),Any%,265.39,2017-12-18,https://www.twitch.tv/videos/210633259,First PB after switching to fast Red Dragon zi...,USA / NTSC,Nintendo Entertainment System,None,baldnate
456,NaN,Strider (NES),Any%,233.99,2018-01-15,https://www.twitch.tv/videos/219078457,"got the air zip. no screen wrap, no yggy quick...",USA / NTSC,Nintendo Entertainment System,None,baldnate
